In [1]:
import numpy as np
import openmm as mm
from openmm.app import *
from openmmtools import respa, utils
from simtk import unit

import polychrom
import os, sys
import polychrom
from polychrom import simulation, starting_conformations, forces, forcekits
import openmm
from polychrom.hdf5_format import HDF5Reporter

In [2]:
from openmmtools.integrators import PrettyPrintableIntegrator

In [ ]:
#What is the monomer diffusion coefficient?
collision_rate = 0.01 * (1 / unit.picosecond)
conlen = 1.0 * unit.nanometer
mass = 100 * unit.amu
temperature = 300
kB = unit.BOLTZMANN_CONSTANT_kB * unit.AVOGADRO_CONSTANT_NA
kT = kB * temperature * unit.kelvin
force = kT / conlen
force * conlen / (mass * collision_rate)

In [51]:
#one way to create a custom integrator is to subclass
class ActiveBrownianIntegrator(utils.RestorableOpenMMObject, PrettyPrintableIntegrator, mm.CustomIntegrator):
    """ Taking inspiration from custom integrators available in openmmtools. They created a
    PrettyPrintableIntegrator class that will print out the computation steps of the integrator.
    
    They also developed a RestorableOpenMMObject: Normally, a custom OpenMM object loses its specific class 
    (and all its methods) when it is copied or deserialized from its XML representation.
    Class interfaces inheriting from this can be restored through the method
    ``restore_interface()``. Also, this class extend the copying functions
    to copy also Python attributes.
    
    """
    def __init__(self, timestep, collision_rate, *args, **kwargs):
        super(ActiveBrownianIntegrator, self).__init__(timestep, *args, **kwargs)
        #define variables
        self.addUpdateContextState()
        self.addGlobalVariable("zeta", collision_rate *(1 / unit.picosecond))
        self.addGlobalVariable("l", 1.0 * unit.nanometer) #length scale of simulation
        self.addPerDofVariable("D", 0.0)
        #update context state before adding any computations
        self.addComputePerDof("D", "f1") #assumes diffusion coefficient is in force group 1
        #WARNING: D is in units of force. To convert to units of diffusion coefficient, 
        #multiply by a length scale and divide by friction coefficient
        #self.addComputePerDof("xi", "zeta*m") #friction coefficient in correct units
        #Euler Marayama update to position
        self.addComputePerDof("v", "v + sqrt(2*(D*l/(zeta*m))/dt)*gaussian")
        #self.addComputePerDof("v", "v + f0/(zeta*m) + sqrt(2*(D*l/(zeta*m))/dt)*gaussian")
        #self.addConstrainVelocities()
        self.addComputePerDof("x", "x + v*dt")
        #self.addConstrainPositions()
        

In [52]:
def diffusion_constant_force(sim_object, D, name="diffusion", force_group=None):
    """ Force to be added to polychrom.forces module.
    Defines a constant external force that varies per particle. Constant force experienced
    by each particle in array D.
    
    Parameters
    ----------
    D : np.ndarray (N,)
        Diffusion coefficients of N particles in units of kT/(mass * collision_rate)
    """
    fakeExternal = mm.CustomExternalForce("fx * (x+y+z)")
    fakeExternal.name = name
    if force_group:
        if type(force_group) != int:
            raise TypeError("Force group should be an integer 0, 1, 2, etc.")
        fakeExternal.setForceGroup(force_group)
    fakeExternal.addPerParticleParameter("fx")
    for i in range(sim_object.N):
        fakeExternal.addParticle(i, [D[i] * sim_object.kT / sim_object.conlen])
        
    return fakeExternal
        

In [64]:
N=100
D = np.tile(1.0, N)
D

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [69]:
reporter = HDF5Reporter(folder="simulations/test_integrator", max_data_length=100, overwrite=True)
collision_rate = 0.01 * (1 / unit.picosecond)
integrator = ActiveBrownianIntegrator(1 * unit.femtosecond, collision_rate)
integrator.pretty_print()
sim = simulation.Simulation(
    platform="CUDA", 
    integrator=integrator,
    error_tol=0.003,
    GPU="1",
    collision_rate=0.01,
    N=N,
    save_decimals=2,
    PBCbox=[50, 50, 50],
    verbose=True,
    reporters=[reporter],
)

polymer = starting_conformations.grow_cubic(N, 50)
sim.set_data(polymer, center=True)  # loads a polymer, puts a center of mass at zero
sim.add_force(diffusion_constant_force(sim, D, force_group=1))

INFO:root:Using the provided integrator object


step      0 : allow forces to update the context state
step      1 : D <- f1
step      2 : v <- v + sqrt(2*(D*l/(zeta*m))/dt)*gaussian
step      3 : x <- x + v*dt


In [70]:
for _ in range(10):  # Do 10 blocks
    sim.do_block(1)  # Of 100 timesteps each. Data is saved automatically. 
sim.print_stats()  # In the end, print very simple statistics

reporter.dump_data()  # always need to run in the end to dump the block cache to the disk


INFO:root:applying forces
INFO:root:adding force diffusion 0
INFO:root:Particles loaded. Potential energy is 0.000000


IntegrationFailError: Coordinates are NANs